In [1]:
#Die ganzen packages die für den versuch verwendet werden

import numpy as np #numpy ist ein package mit allen mathematischen ausdrücken/funktionen/operationen/etc. 
from uncertainties import ufloat #Uncertainties ist für das automatische berrechnen von unsicherheiten
from uncertainties.umath import * #Importieren von rechnen mit uncertainties (addition/multiplikation/etc.)
from uncertainties import unumpy as unp #kann arrays von unsicherheiten verstehen (also eine liste, statt jede zahl einzeln einzutippen)
from IPython.display import Latex, HTML, Math, display # Um ergebnisse schöner anzuzeigen. display() "ersetzt" quasi print(), und lässt latex fromatierung zu
import matplotlib.pyplot as plt #Damit kann man graphen plotten

import pandas as pd #Um die Spektroskopie daten zu lesen


#funktion für mittwlert mit fehler defnieren (um später ein uarray mit unsicherheiten direkt zu einem mittelwert zu machen)
def mean(values):
    nom = unp.nominal_values(values)
    std_mean = np.std(nom, ddof=1) / np.sqrt(len(nom))
    return ufloat(np.mean(nom), std_mean)

# Brechung, Dispersion und Spektroskopie

## Dispersionskurve eines optischen Glases

Es wird die Disperionskurve eines Optischen Glases mittels Spektroskopie bestimmt.

### Grundlagen

### Durchführung und Aufbau

### Ergebnisse

Die mittels Goniometer bestimmten Minimal Winkel und den Dementsprechenden Brechungsindizes sind in Tabelle 1. angeführt.

In [2]:
#Goniometer: Spektrallinien einer Quecksilberlampe

#Spektrallinien Winkel:

# Unsicherheit vom winkel nach augenmaß (höher als auflösung vom goniometer = 0.01)
Winkel_std = 0.1
# Epsilon winkel vom Prisma:
e = 60

#                          Lila     Blau   Grün    Gelb
ungebrochen = unp.uarray([ 90.64 , 91.73 , 92.10 , 92.95 ], Winkel_std)
gebrochen   = unp.uarray([ 41.31 , 41.96 , 42.48 , 43.05 ], Winkel_std)

delta_min = ungebrochen - gebrochen
n = unp.sin((delta_min + e)/2 * np.pi/180) / unp.sin(e/2 * np.pi/180) # * np.pi/180 weil np.sin nur mit Bogenmaß arbeitet (!!!nicht mit winkelmaß!!!)


#----------------------------------------------------------------------------------------------------------------------------------------------
#
# Ab hier alles nur um die Tabellen schön darzustellen. Man könnte auch einfach print(n) und print(delta_min) machen ist aber uuuurrrr schirch
#
#----------------------------------------------------------------------------------------------------------------------------------------------


meine_lieblingsfarben = (["Lila", "Blau", "Grün", "Gelb"])

#Minimum winkel als Tabelle anzeigen
table = r"""$$
\begin{array}{c|c|c}
\textbf{Farbe} & \textbf{Min. Winkel} ~\delta_{min}~\mathrm[°] & \textbf{Brechungsindex} ~n~ \\ \hline \\
"""
for i, (farbe,Delta,n) in enumerate(zip(meine_lieblingsfarben, delta_min, n)):
    table += rf"\mathrm{{{farbe}}} & ({Delta:.2f}) & ({n:.3f}) \\ "
table += r"\end{array}$$"

display(Latex(table))

<IPython.core.display.Latex object>

Spektrometer blablablbalbalbalb

In [3]:
#Spektrum der Quecksilberdampflampe


Spektrum = pd.read_csv("GPT-Data.csv", sep=',', decimal='.')


wellenlänge_wert = Spektrum['Wavelength(nm)'].values
wellenlänge = wellenlänge_wert
Int_1 = Spektrum['Intenstity'].values


plt.figure(figsize=(7, 5))

plt.plot(wellenlänge,Int_1, 'o', markersize=2,zorder=10,label="Rote LED", color='Red')

plt.xlabel("$U$ [V]")
plt.ylabel("$I$ [A]")
plt.title("Strom-Spannungs-Kennlinie LED")
plt.ylim(bottom=-0.1)
plt.xlim(left=0)
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("/mnt/c/Physik/Laborprakitkum2/PS10/Kennlinie_LED.pdf")
#plt.savefig("/mnt/f/Physik studium/LP2/PS10/Kennlinie_LED.pdf")
plt.show()


#Wellenlängen - Man braucht höhere operating voltages desto kleiner die wellenlänge wird (Energie steigt).

plt.figure(figsize=(7, 5))


# Maxima finden und vertikale Linien einzeichnen
farben = ['orange', 'blue', 'green', 'red']
intensitäten = [ Int_4, Int_1, Int_2, Int_3]
labels = ['Weiße LED', 'Blaue LED', 'Grüne LED', 'Rote LED']

# Plot Kurven
for intensität, farbe, label in zip(intensitäten, farben, labels):
    plt.plot(wellenlänge, intensität, '-', markersize=3, zorder=10, label=label, color=farbe)

    # Maxima
    max_idx = np.argmax(intensität)
    λ_max = wellenlänge[max_idx]
    I_max = intensität[max_idx]

    # Vertikale gestrichelte Linie bei Maximum
    plt.vlines(x=λ_max, ymin=0, ymax=I_max, linestyles='dashed', colors=farbe)
    plt.plot(λ_max, I_max, 'o', color=farbe, markersize=5)
    plt.text(λ_max-10, I_max+100, f'{int(λ_max)} nm', color=farbe, fontsize=8)
    
    # Halbwertshöhe
    halbwert = I_max / 2


    # Interpolation links des Maximums
    links = np.where(wellenlänge < λ_max)[0]
    for i in reversed(links):
        if intensität[i] <= halbwert:
            λ_links = np.interp(halbwert, [intensität[i], intensität[i+1]], [wellenlänge[i], wellenlänge[i+1]])
            break

    # Interpolation rechts des Maximums
    rechts = np.where(wellenlänge > λ_max)[0]
    for i in rechts:
        if intensität[i] <= halbwert:
            λ_rechts = np.interp(halbwert, [intensität[i-1], intensität[i]], [wellenlänge[i-1], wellenlänge[i]])
            break
    
    # FWHM berechnen
    fwhm = λ_rechts - λ_links
    unsicherheit = fwhm / 2

    # FWHM Bereich markieren
    plt.hlines(y=halbwert, xmin=λ_links, xmax=λ_rechts, colors='black', linestyles='solid',zorder=11)

    # Vertikale Ticks links und rechts
    tick_height = 50
    plt.vlines(x=λ_links, ymin=halbwert - tick_height, ymax=halbwert + tick_height, color='black', linewidth=1,zorder=11)
    plt.vlines(x=λ_rechts, ymin=halbwert - tick_height, ymax=halbwert + tick_height, color='black', linewidth=1,zorder=11)

    
    # Ausgabe: λ ± Δλ als Math-Formel
    display(Math(rf"\lambda_{{\mathrm{{{label}}}}} = {λ_max:.2f} \pm {unsicherheit:.2f}\ \mathrm{{nm}}"))
    display(Math(rf"F = {fwhm:.2f} \mathrm{{nm}}"))
plt.plot([0,0],[0,0], '-', color="black", label="FWHM")
plt.xlabel("$Wellenlänge$ [nm]")
plt.ylabel("$Intesnität$ [counts]")
plt.title("Spektrum LED")
plt.ylim(bottom=0, top=4000)
plt.xlim(left=400, right=750)
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("/mnt/c/Physik/Laborprakitkum2/PS10/LED_spektrum.pdf")
#plt.savefig("/mnt/f/Physik studium/LP2/PS10/LED_spektrum.pdf")
plt.show()


KeyError: 'Intenstity'